In [ ]:
!pip install -U openai-whisper
!pip install gradio

In [1]:
import whisper
import IPython
import gradio as gr 
import time

In [2]:
model = whisper.load_model("base")

In [ ]:
audio_path = "https://github.com/jesus-seijas-sp/workshopnlp/raw/main/data/audio01.mp3"

In [3]:
IPython.display.Audio(audio_path)

In [4]:
audio = whisper.load_audio(audio_path)
audio = whisper.pad_or_trim(audio)

In [5]:
mel = whisper.log_mel_spectrogram(audio).to(model.device)
mel

tensor([[-0.5504, -0.5504, -0.5504,  ..., -0.5504, -0.5504, -0.5504],
        [-0.5504, -0.5504, -0.5504,  ..., -0.5504, -0.5504, -0.5504],
        [-0.5504, -0.5504, -0.5504,  ..., -0.5504, -0.5504, -0.5504],
        ...,
        [-0.5504, -0.5504, -0.5504,  ..., -0.5504, -0.5504, -0.5504],
        [-0.5504, -0.5504, -0.5504,  ..., -0.5504, -0.5504, -0.5504],
        [-0.5504, -0.5504, -0.5504,  ..., -0.5504, -0.5504, -0.5504]],
       device='cuda:0')

In [6]:
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: en


In [7]:
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)
print(result)

DecodingResult(audio_features=tensor([[-1.7324,  0.1509, -0.2488,  ..., -0.2805,  0.3623,  0.0660],
        [-0.8428,  0.0725,  1.1191,  ...,  0.8984, -0.0168, -0.0129],
        [-0.3220,  0.3137,  0.9102,  ...,  1.1426, -0.0034,  0.0348],
        ...,
        [-0.1370, -1.1797,  0.5894,  ...,  0.5503, -0.1980,  0.5264],
        [-0.5571, -1.0508,  0.4155,  ...,  0.1086,  0.3936,  0.0705],
        [-1.0918, -0.7188,  0.8711,  ...,  0.2428, -1.1553, -1.1396]],
       device='cuda:0', dtype=torch.float16), language='en', language_probs=None, tokens=[50364, 6059, 632, 257, 707, 10097, 11, 1080, 25146, 384, 390, 2418, 382, 5756, 11, 293, 5315, 300, 6059, 1437, 11, 264, 50712, 50712, 10097, 390, 988, 281, 352, 13, 50774], text='Mary had a little lamb, its fleece was white as snow, and everywhere that Mary went, the lamb was sure to go.', avg_logprob=-0.21724994480609894, no_speech_prob=0.0410284660756588, temperature=0.0, compression_ratio=1.211111111111111)


In [8]:
print(result.text)

Mary had a little lamb, its fleece was white as snow, and everywhere that Mary went, the lamb was sure to go.


In [9]:
to_english = model.transcribe(audio, task = 'translate')
print(to_english["text"])

 Mary had a little lamb, its fleece was white as snow, and everywhere that Mary went, the lamb was sure to go.


In [10]:
def transcribe(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    to_english = model.transcribe(audio, task = 'translate')
    return result.text, to_english['text']

In [11]:
gr.Interface(
    title = 'Talk to me', 
    fn=transcribe, 
    inputs=[gr.inputs.Audio(source="microphone", type="filepath")],
    outputs=["textbox"],
    live=True).launch()

C:\Users\jseij\conda3\lib\site-packages\gradio\inputs.py:318: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\jseij\conda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Hint: Set streaming=True for Audio component to use live streaming.
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\jseij\conda3\lib\site-packages\gradio\processing_utils.py:225: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Detected language: fr
